# facts
Signatures are continuous segments of chromosomes \
Genes can (and often do) belong to multiple signatures \


In [1]:
import pandas as pd
import pathlib
import os
import json
import itertools
import numpy as np
import functools
import shelve

root = pathlib.Path("/data/rbg/shared/datasets/TCGA/")


def wrap_function_with_disk_cache(func, cache_file):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        key_str = str(key)
        with shelve.open(cache_file) as db:
            if key_str in db:
                return db[key_str]
            result = func(*args, **kwargs)
            db[key_str] = result
            return result
    return wrapper


In [2]:

def add_outcome(elem):
    vital_status = elem['demographic'].get('vital_status')

    if vital_status == 'Dead':
        patient_death = elem['demographic'].get('days_to_death')
        return [1, int(patient_death)]

    elif vital_status == "Alive":
        follow_ups = elem['follow_ups']
        follow_up_days = [
            f.get('days_to_follow_up')
            for f in follow_ups
            if f.get('days_to_follow_up') is not None
        ]
        diagnosis_days = [
            d.get('days_to_last_follow_up')
            for d in elem['diagnoses']
            if d.get('days_to_last_follow_up') is not None
        ]
        all_days = follow_up_days + diagnosis_days
        date_of_last_checkup = max(all_days) if all_days else None

        return [0, int(date_of_last_checkup)]
    else:
        typing.assert_never("vital status not in Alive/Dead")


def remove_sparse_features(X, threshold=0.05):
    non_zero_fraction = (X != 0).sum(axis=0) / X.shape[0]
    keep = non_zero_fraction > threshold
    return X.loc[:, keep]


def remove_perfect_correlations(X):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] == 1)]
    return X.drop(columns=to_drop)


# ids = pd.read_json("ids/tcga_cohort.json")["case_id"]
ids = pd.read_csv("sample_ids.csv")["Sample"]

case_ids_raw = pd.read_json("./clinical-tcga-brca.json").set_index("submitter_id")
case_ids = case_ids_raw.loc[ids]
case_ids["outcome"] = case_ids.apply(add_outcome, axis=1)

case_ids = case_ids.reset_index()
print(case_ids.shape)

case_ids = case_ids.drop_duplicates(subset=["case_id"])
print(case_ids.shape)

case_ids.head()


(133, 16)
(133, 16)


,submitter_id,disease_type,project,days_to_consent,diagnoses,consent_type,demographic,primary_site,updated_datetime,case_id,follow_ups,index_date,state,lost_to_followup,exposures,outcome
0,TCGA-A1-A0SP,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},55.0,"[{'tissue_or_organ_of_origin': 'Breast, NOS', ...",Informed Consent,{'demographic_id': 'f7ed8691-ed3d-54bf-94c5-7f...,Breast,2025-01-06T07:38:56.031656-06:00,a9bb8159-32f0-454c-a946-b3286a52b9d5,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[0, 584]"
1,TCGA-A2-A04P,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},22.0,"[{'morphology': '8500/3', 'submitter_id': 'TCG...",Informed Consent,{'demographic_id': '92faa22e-8e19-5502-987c-a6...,Breast,2025-01-06T07:38:16.470569-06:00,ccd4a24b-d8cc-4686-9dee-c98b0c5a8d21,"[{'days_to_progression': 102, 'timepoint_categ...",Diagnosis,released,No,NaN,"[1, 548]"
2,TCGA-A2-A04T,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},29.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': 'dd68f2a1-f279-567f-83b6-4c...,Breast,2025-01-05T21:21:42.777520-06:00,b58ad350-5140-4fa8-bc2c-24bca8395f3a,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[0, 2246]"
3,TCGA-A2-A04U,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},23.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '9145b76c-61b5-56e6-876e-58...,Breast,2025-01-05T23:51:14.306282-06:00,1c3610f7-e0aa-48d7-9a27-0dbaf6e244f9,"[{'timepoint_category': 'Follow-up', 'follow_u...",Diagnosis,released,Yes,NaN,"[0, 2654]"
4,TCGA-A2-A0CM,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},-212.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '0f8728a4-3abb-5a78-84c3-3d...,Breast,2025-01-06T07:33:56.519794-06:00,eb2dbb4f-66b6-4525-8323-431970f7a64e,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[1, 754]"


In [6]:
dna_mapping = "/data/rbg/shared/datasets/TCGA/Genomic/mutations.gdc_sample_sheet.2025-07-07.tsv"

caseid_mut_mapping = pd.read_csv(dna_mapping, delimiter='\t')

caseid_mut_mapping['Case ID'] = caseid_mut_mapping['Case ID'].str.split(',')
# print(caseid_mut_mapping.shape)
caseid_mut_mapping = caseid_mut_mapping.explode('Case ID')
caseid_mut_mapping['Case ID'] = caseid_mut_mapping['Case ID'].str.strip()
# print(caseid_mut_mapping.shape)
caseid_mut_mapping = caseid_mut_mapping.drop_duplicates(subset=["Case ID"]) # I'm pretty sure this is valid bc I explode only on Case ID - meaning everything else is duplicated. This whole process might actualyl be unncesary. TODO: check if all "Case IDs pairs" are just twins
print(caseid_mut_mapping.shape)


# # stupid test to check whether any of hte records had duplicate sample IDs
# # def foo(x):
# #     a = list(map(lambda x: x.strip(), x["Sample ID"].split(",")))
# #     print(a[0])
# #     print(a[1])
# #     # return a[0] == a[1]
# # caseid_copynum_mapping = caseid_copynum_mapping[caseid_copynum_mapping.apply(foo, axis=1)]

caseid_mut_mapping.head()

(2774, 11)


,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Tissue Type,Tumor Descriptor,Specimen Type,Preservation Method
0,0518551d-4df2-4124-b68d-494200c5586b,e75893cc-346b-4567-bdd4-f88f3444a72a.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-BRCA,TCGA-E2-A158,"TCGA-E2-A158-01A, TCGA-E2-A158-10A","Tumor, Normal","Primary, Not Applicable","Solid Tissue, Peripheral Blood NOS","Unknown, Unknown"
2,98953412-ae65-4116-b43e-20278294e4f2,TCGA_BRCA.a5279f5e-626b-45ca-86fa-976ae328fa2b...,Simple Nucleotide Variation,Annotated Somatic Mutation,TCGA-BRCA,TCGA-E9-A1R0,"TCGA-E9-A1R0-10A, TCGA-E9-A1R0-01A","Normal, Tumor","Not Applicable, Primary","Peripheral Blood NOS, Solid Tissue","Unknown, OCT"
3,3acd8060-5f26-4d22-b318-1b2427e1e30c,54f9af00-9550-45a5-b3c4-3af171e0300a.wxs.VarSc...,Simple Nucleotide Variation,Annotated Somatic Mutation,TCGA-BRCA,TCGA-C8-A27B,"TCGA-C8-A27B-10A, TCGA-C8-A27B-01A","Normal, Tumor","Not Applicable, Primary","Peripheral Blood NOS, Solid Tissue","Unknown, OCT"
5,4a3fc24a-c86b-48e0-bc11-c91fcc09a317,308c1b03-8cf4-44a5-aa18-12f9e965d85b.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-BRCA,TCGA-AN-A03Y,"TCGA-AN-A03Y-10A, TCGA-AN-A03Y-01A","Normal, Tumor","Not Applicable, Primary","Peripheral Blood NOS, Unknown","Unknown, OCT"
6,938979df-2611-4826-9982-8cfc0395e5dd,efb68702-7e0e-474d-a6b5-2855d963e3da.wxs.aliqu...,Simple Nucleotide Variation,Aggregated Somatic Mutation,TCGA-BRCA,TCGA-B6-A0WW,"TCGA-B6-A0WW-01A, TCGA-B6-A0WW-10A","Tumor, Normal","Primary, Not Applicable","Solid Tissue, Peripheral Blood NOS","OCT, Unknown"


In [8]:
# add copynumbesr to "IDs" dataset
# FIXME: figure out why 21 records are being dropped - the final result should still be 133
print(case_ids.shape)
data= pd.merge(
    case_ids,
    caseid_mut_mapping[['Case ID', 'Sample ID', 'File ID', "File Name"]],  # select only the needed column
    left_on='submitter_id',
    right_on='Case ID',
    how="inner"
)
print(data.shape)

(133, 16)
(133, 20)


In [23]:
# !pip install BGZF

# !zcat /data/rbg/shared/datasets/TCGA/Genomic/downloads/49fe1ce1-3dc8-4130-a29c-3f3ca82bfe61/c30ef13b-f23a-46c2-9187-f64269318358.wxs.pindel.raw_somatic_mutation.vcf.gz | head -20
# !file /data/rbg/shared/datasets/TCGA/Genomic/downloads/49fe1ce1-3dc8-4130-a29c-3f3ca82bfe61/c30ef13b-f23a-46c2-9187-f64269318358.wxs.pindel.raw_somatic_mutation.vcf.gz

os.listdir(f"{root}/Genomic/downloads/{elem['File ID']}")

['logs',
 'c30ef13b-f23a-46c2-9187-f64269318358.wxs.pindel.raw_somatic_mutation.vcf.gz',
 'c30ef13b-f23a-46c2-9187-f64269318358.wxs.pindel.raw_somatic_mutation.vcf.gz.tbi']

In [74]:
import pysam
import pyranges as pr
import pandas as pd
import requests
import time

elem = data.iloc[8]
path = f"{root}/Genomic/downloads/{elem['File ID']}/{elem['File Name']}"
print(path)
assert (path.endswith(".gz"))

vcf = pysam.VariantFile(path)

import pysam
import pandas as pd

elem = data.iloc[8]
path = f"{root}/Genomic/downloads/{elem['File ID']}/{elem['File Name']}"
print(path)
assert path.endswith(".gz")

vcf = pysam.VariantFile(path)

variants = []

for record in vcf.fetch():
    chrom = record.chrom
    pos = record.pos
    var_id = record.id
    ref = record.ref
    alt = ','.join(str(a) for a in record.alts)
    qual = record.qual
    
    filters = list(record.filter.keys()) if record.filter else []
    
    # INFO fields - somatic flag, CSQ annotation, allele counts
    is_somatic = 'SOMATIC' in record.info
    
    csq = record.info.get('CSQ')
    # csq_fields = csq.split(',')[0].split('|') if csq else []
    csq_fields = list(csq)
    
    # Extract some common VEP fields by their position in the CSQ format:
    # Format is documented in the header, but common fields: 
    # Allele(0),Consequence(1),IMPACT(2),SYMBOL(3),Gene(4),Feature_type(5),Feature(6),BIOTYPE(7),HGVSc(10),HGVSp(11),CLIN_SIG(66) etc.
    # Adjust indices if needed based on your actual header
    consequence = csq_fields[1] if len(csq_fields) > 1 else None
    impact = csq_fields[2] if len(csq_fields) > 2 else None
    gene = csq_fields[4] if len(csq_fields) > 4 else None
    clin_sig = csq_fields[66] if len(csq_fields) > 66 else None
    
    # FORMAT fields for each sample
    # Assume samples named "NORMAL" and "TUMOR" as per header
    normal_sample = record.samples.get('NORMAL')
    tumor_sample = record.samples.get('TUMOR')
    
    # Genotype
    normal_gt = normal_sample['GT'] if normal_sample else None
    tumor_gt = tumor_sample['GT'] if tumor_sample else None
    
    normal_dp = normal_sample['DP'] if normal_sample and 'DP' in normal_sample else None
    tumor_dp = tumor_sample['DP'] if tumor_sample and 'DP' in tumor_sample else None
    
    # Allelic depths (AD), tuple or list per REF and ALT
    normal_ad = normal_sample['AD'] if normal_sample and 'AD' in normal_sample else None
    tumor_ad = tumor_sample['AD'] if tumor_sample and 'AD' in tumor_sample else None
    
    variants.append({
        'chrom': chrom,
        'pos': pos,
        'id': var_id,
        'ref': ref,
        'alt': alt,
        'qual': qual,
        'filters': filters,
        'somatic': is_somatic,
        'consequence': consequence,
        'impact': impact,
        'gene': gene,
        'clin_sig': clin_sig,
        'normal_gt': normal_gt,
        'tumor_gt': tumor_gt,
        'normal_dp': normal_dp,
        'tumor_dp': tumor_dp,
        'normal_ad': normal_ad,
        'tumor_ad': tumor_ad,
    })

variants_df = pd.DataFrame(variants)
variants_df.head()


/data/rbg/shared/datasets/TCGA/Genomic/downloads/dd7e8bb1-e084-4b8a-bf3b-1064de7444d8/TCGA_BRCA.3c9305f7-9ef0-44f4-8caa-c73e689ad319.wxs.MuSE.somatic_annotation.vcf.gz
/data/rbg/shared/datasets/TCGA/Genomic/downloads/dd7e8bb1-e084-4b8a-bf3b-1064de7444d8/TCGA_BRCA.3c9305f7-9ef0-44f4-8caa-c73e689ad319.wxs.MuSE.somatic_annotation.vcf.gz


,chrom,pos,id,ref,alt,qual,filters,somatic,consequence,impact,gene,clin_sig,normal_gt,tumor_gt,normal_dp,tumor_dp,normal_ad,tumor_ad
0,chr1,1022338,None,G,A,None,[PASS],True,A|5_prime_UTR_variant|MODIFIER|AGRN|ENSG000001...,None,None,None,"(0, 0)","(0, 1)",19,170,"(19, 0)","(119, 50)"
1,chr1,1750064,None,T,A,None,[PASS],True,A|downstream_gene_variant|MODIFIER|NADK|ENSG00...,A|downstream_gene_variant|MODIFIER|NADK|ENSG00...,A|downstream_gene_variant|MODIFIER|NADK|ENSG00...,None,"(0, 0)","(0, 1)",8,12,"(8, 0)","(9, 3)"
2,chr1,2390118,None,C,T,None,[PASS],True,T|upstream_gene_variant|MODIFIER|RER1|ENSG0000...,T|upstream_gene_variant|MODIFIER|RER1|ENSG0000...,T|intron_variant&NMD_transcript_variant|MODIFI...,None,"(0, 0)","(0, 1)",34,51,"(34, 0)","(35, 16)"
3,chr1,2652391,None,G,C,None,[Tier1],True,C|intron_variant|MODIFIER|TTC34|ENSG0000021591...,C|regulatory_region_variant|MODIFIER|||Regulat...,None,None,"(0, 0)","(0, 1)",30,47,"(29, 1)","(45, 2)"
4,chr1,2653973,None,C,G,None,[PASS],True,G|intron_variant|MODIFIER|TTC34|ENSG0000021591...,None,None,None,"(0, 0)","(0, 1)",70,132,"(69, 0)","(124, 8)"


In [63]:
!file $path
print(os.listdir(f"{root}/Genomic/downloads/{elem['File ID']}"))

/data/rbg/shared/datasets/TCGA/Genomic/downloads/dd7e8bb1-e084-4b8a-bf3b-1064de7444d8/TCGA_BRCA.3c9305f7-9ef0-44f4-8caa-c73e689ad319.wxs.MuSE.somatic_annotation.vcf.gz: Blocked GNU Zip Format (BGZF; gzip compatible), block length 11659
['logs', 'TCGA_BRCA.3c9305f7-9ef0-44f4-8caa-c73e689ad319.wxs.MuSE.somatic_annotation.vcf.gz', 'TCGA_BRCA.3c9305f7-9ef0-44f4-8caa-c73e689ad319.wxs.MuSE.somatic_annotation.vcf.gz.tbi']


In [76]:
import subprocess
import pathlib
import os

def run_vcf2maf(vcf_path, maf_output, ref_fasta, vep_path, vep_cache):
    """
    Convert VCF to MAF using vcf2maf.pl wrapper.

    Args:
        vcf_path (str): Input VCF file path (gzipped recommended).
        maf_output (str): Output MAF file path.
        ref_fasta (str): Path to reference fasta (e.g. GRCh38.fa).
        vep_path (str): Path to VEP executable.
        vep_cache (str): Path to VEP cache directory.
    """
    # Command template for vcf2maf.pl
    cmd = [
        "vcf2maf.pl",
        "--input-vcf", vcf_path,
        "--output-maf", maf_output,
        "--ref-fasta", ref_fasta,
        "--vep-path", vep_path,
        "--vep-data", vep_cache,
        "--tumor-id", "TUMOR",      # Adjust sample IDs as needed
        "--normal-id", "NORMAL"
    ]

    print("Running vcf2maf with command:")
    print(" ".join(cmd))

    try:
        subprocess.run(cmd, check=True)
        print(f"MAF successfully generated: {maf_output}")
    except subprocess.CalledProcessError as e:
        print(f"vcf2maf failed: {e}")

# export VCF2MAF_URL=`curl -sL https://api.github.com/repos/mskcc/vcf2maf/releases | grep -m1 tarball_url | cut -d\" -f4`
# curl -L -o mskcc-vcf2maf.tar.gz $VCF2MAF_URL; tar -zxf mskcc-vcf2maf.tar.gz; cd mskcc-vcf2maf-*
# mkdir vcf2maf_cache

# git clone https://github.com/Ensembl/ensembl-vep.git
# cd ensemble-vep.git
# cpan DBI
# perl INSTALL.pl


print(pathlib.Path(path).with_suffix("maf"))

# if __name__ == "__main__":
#     vcf_path = path
#     maf_output = 
#     ref_fasta = "GRCh38.primary_assembly.genome.fa.gz"
#     vep_path = "./vcf2maf/vcf2maf.pl"
#     vep_cache = "./vcf2maf_cache/"

#     run_vcf2maf(vcf_path, maf_output, ref_fasta, vep_path, vep_cache)

NameError: name 'Pathlib' is not defined